## Aprendizagem Automática - Projeto

In [7]:
# Imports
import pandas as pd

# Models and selection methods
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
# Linear regression metrics
from sklearn.metrics import explained_variance_score, mean_squared_error, max_error, mean_absolute_error
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.preprocessing import LabelBinarizer

In [8]:
# Funcoes auxiliares

def printRegStatistics(truth, preds):
    print("The RVE is: ", explained_variance_score(truth, preds))
    print("The rmse is: ", mean_squared_error(truth, preds, squared=False))
    corr, pval = pearsonr(truth, preds)
    print("The Correlation Score is is: %6.4f (p-value=%e)\n"%(corr,pval))
    print("The Maximum Error is is: ", max_error(truth, preds))
    print("The Mean Absolute Error is: ", mean_absolute_error(truth, preds))
    

#Devolve uma lista com quantos elementos há seguidos entre valores de NaN. p.e: [1,nan,2,3,nan,nan,2,nan,2,3,4] -> [1,2,1,3]
def count_not_nan(serie):
    conta=0
    lista_seguida=[]
    for i in range(len(serie)):
        if serie[i]==1:
            conta+=1
        else:
            if conta!=0:
                lista_seguida.append(conta)
                conta=0
            else:
                conta=0
                
        if i==len(serie)-1 and conta!=0:
            lista_seguida.append(conta)
    return lista_seguida

#Função que lida com os nan nas df. Se o país não tiver nenhum dado -> drop. Se o país não tiver pelo menos lag+1 dados seguidos
#-> drop. O resto fica

def drop_nan(df,lag):
    pais_nan=[]
    for pais in df["Country Name"].index:
        nan_val=list(pd.DataFrame(df.iloc[pais][4:]).count(axis=1)) #ver se os valores são nan
        if sum(nan_val)==0: #se for tudo nan
            pais_nan.append(pais)
        elif max(count_not_nan(nan_val))<=lag: #Se não houver mais do que lag valores seguidos, drop
            pais_nan.append(pais)

    df=df.drop(pais_nan, axis=0)
    df = df.reset_index(drop=True)
    return df

#Função que escolha o dataframe até um certo ano apenas, que remove os países que contenham nan nesse intervalo de tempo

def indexing(dataf, ano_i):
    X_list=dataf.drop(columns=dataf.iloc[:,[1,2,3]]+dataf.loc[:,[str(i) for i in range(1960,ano_i)]])
    X_list=X_list.T
    X_list=X_list.rename(columns=X_list.iloc[0]).drop(X_list.index[0])
    return X_list

#Cria um DataFrame com os dados de um país e respetivos lags

def lags(df, lag, país):
    new_df=pd.DataFrame()
    new_df["dados"]=df[país]
    colunas = [str(i) + "lag" for i in range(1,lag+1)]

    for i,k in enumerate(colunas):
        new_df[k] = new_df["dados"].shift(+i+1)
    new_df=new_df.dropna()
    return new_df

#Função que cria o dataset necessário para as timeseries. As time series são todas unidas numa só coluna e os países respetivos
# a essa série temporal recebem um 1. Também concatena os lags no início da df.

def time_series(df, lag):
    binario = LabelBinarizer().fit(df.columns)
    df_final= pd.DataFrame()
    for pais in df.columns:
        pais_bin=pd.DataFrame(data=list(binario.transform([pais]))*len(df[pais]),
                             columns=binario.classes_,
                             index=df.index)
        #Agora juntamos os lags e os dados respetivos
        df_mid=pd.DataFrame()
        df_mid=pd.concat((df_mid,lags(df, lag, pais),pais_bin),axis=1).dropna()

        #Finalmente, concatenamos tudo numa só DataFrame
        df_final = pd.concat((df_final, df_mid), sort=True)
        df_fina=df_final.dropna()
    return df_final

# Função para criar um dataframe X para um dado dataset, pais e ano, este dataframe depois eh utilizado para fazer predict
def createX(dataset : pd.DataFrame, country : str, year : int, lag) -> pd.DataFrame:
    previousYearCountryData = dataset.loc[(dataset[country] == 1) & (dataset.index.get_level_values(0) == str(year - 1))]
    newX = previousYearCountryData.copy()
    newX.index = [str(year)]
    for i in range(lag,2,-1):
        newX.at[str(year), str(i)+"lag"] = newX.at[str(year),str(i-1)+"lag"]
    newX.at[str(year), "1lag"] = newX.at[str(year),"dados"]
    newX.drop(["dados"], axis=1, inplace=True)
    return newX

# Função para prever dados novos para um dado dataset ano e pais a partir do modelo treinado dado
def predictAndCalculateDelta(model, dataset : pd.DataFrame, country : str, year : int, lag):
    previousYearCountryData = dataset.loc[(dataset[country] == 1) & (dataset.index.get_level_values(0) == str(year - 1))]
    newX = createX(dataset, country, year, lag)
    x = model.predict(newX)
    x_previous = previousYearCountryData.at[str(year-1),"dados"]
    delta = x - x_previous
    return x[0], delta[0]

# Vai buscar o valor para um dado ano e pais e calcula o delta
def determineRealResults(dataset : pd.DataFrame, country : str, year : int):
    real_data = dataset.at[str(year), country]
    previous_year_data = dataset.at[str(year-1), country]
    delta = real_data - previous_year_data
    return real_data, delta

# Erro relativo e absoluto
def calculateErrors(real, pred):
    abs_error = abs(pred - real)
    relative_error = abs_error / abs(real)
    return abs_error, (relative_error*100)

# Criar dataframe com os resultados finais
def DFCreate(PrevisionData, year):
    # Criar dataframe
    df = pd.DataFrame(PrevisionData, columns=['Country', str(year) + ' Real Value', str(year) + ' Prevision','Relative Error %',str(year) + ' Real Delta',str(year) + ' Delta Prevision','Relative Error %'])
    # Converter valores em notacao legivel
    df = df.loc[:, df.columns == 'Country'].join(df.loc[:, df.columns != 'Country'].applymap(lambda x: '{:,.3f}'.format(x)))
    return df

Importação dos datasets

In [9]:
# Dados 1960-2016
fertility = pd.read_csv("fertility_rate.csv")
population= pd.read_csv("country_population.csv")
expectancy = pd.read_csv("life_expectancy.csv")

fertility=drop_nan(fertility,4)
population=drop_nan(population,4)
expectancy=drop_nan(expectancy,4)

# Dados 1960-2020
fertility_2020 = pd.read_csv("fertility_2020.csv")
population_2020 = pd.read_csv("population_2020.csv")
expectancy_2020 = pd.read_csv("life_2020.csv")

#Criar os data frames
fert=indexing(fertility, 1960)
pop=indexing(population, 1960)
exp=indexing(expectancy, 1960)

fert_2020 = indexing(fertility_2020, 1960)
pop_2020 = indexing(population_2020, 1960)
exp_2020 = indexing(expectancy_2020, 1960)

Vamos continuar a transformar o nosso dataset, de forma a que fiquem todas as séries temporais numa só coluna, a referência respetiva de que país se refere a série temporal e os respetivos lags para cada valor. 

In [10]:
#Versão final dos datasets
pop_fin=time_series(pop,4)
fert_fin=time_series(fert,4)
exp_fin=time_series(exp,4)

# Divisao em X_train, X_test e y_train, y_test
y_pop = pop_fin["dados"]
X_pop = pop_fin[list(pop_fin.columns)[:-1]]
X_pop_train, X_pop_test, y_pop_train, y_pop_test = train_test_split(X_pop, y_pop, test_size = 0.3, random_state=314)

y_fert = fert_fin["dados"]
X_fert = fert_fin[list(fert_fin.columns)[:-1]]
X_fert_train, X_fert_test, y_fert_train, y_fert_test = train_test_split(X_fert, y_fert, test_size = 0.3, random_state=314)

y_exp = exp_fin["dados"]
X_exp = exp_fin[list(exp_fin.columns)[:-1]]
X_exp_train, X_exp_test, y_exp_train, y_exp_test = train_test_split(X_exp, y_exp, test_size = 0.3, random_state=314)

Com isto feito, podemos passar ao treino dos modelos

## Escolha dos parametros e treino de dos modelos

### Populacao

Treino de 2 modelos, LinearRegression e DecisionTreeRegressor

In [25]:
# LinearRegression
params_linear = [
    {"alpha" : [x*0.1 for x in range(1,10)]}]

grid_search_lasso_pop = GridSearchCV(
    Lasso(), params_linear, scoring="neg_mean_squared_error", cv=15, n_jobs=-1)

grid_search_lasso_pop.fit(X_pop_train, y_pop_train)
print(grid_search_lasso_pop.best_params_)
print("Melhores Parâmetros:", grid_search_lasso_pop.best_params_,"\n")
preds= grid_search_lasso_pop.predict(X_pop_test)
printRegStatistics(y_pop_test, preds)

{'alpha': 0.9}
Melhores Parâmetros: {'alpha': 0.9} 

The RVE is:  0.9999941495255327
The rmse is:  1630219.0785450833
The Correlation Score is is: 1.0000 (p-value=0.000000e+00)

The Maximum Error is is:  12067018.727499008
The Mean Absolute Error is:  537591.1326022077


In [26]:
# DecisionTreeRegressor
params =[{"max_depth" : [4,8,12,15,20]}]

grid_search_dtr_pop = GridSearchCV(
    DecisionTreeRegressor(), params, scoring="neg_mean_squared_error", cv=15, n_jobs=-1)


grid_search_dtr_pop.fit(X_pop_train, y_pop_train)
print(grid_search_dtr_pop.best_params_)
print("Melhores Parâmetros:", grid_search_dtr_pop.best_params_,"\n")
preds= grid_search_dtr_pop.predict(X_pop_test)
printRegStatistics(y_pop_test, preds)

{'max_depth': 12}
Melhores Parâmetros: {'max_depth': 12} 

The RVE is:  0.9999196333726621
The rmse is:  6041993.192661398
The Correlation Score is is: 1.0000 (p-value=0.000000e+00)

The Maximum Error is is:  106640991.0
The Mean Absolute Error is:  1323007.3992482505


In [39]:
# Melhor modelo eh o DecisionTreeRegressor
pop_model = grid_search_lasso_pop

### Fertility

Treino de 2 modelos, LinearRegression e DecisionTreeRegressor

In [28]:
# LinearRegression
params_linear = [
    {"alpha" : [x*0.1 for x in range(1,10)]}]

grid_search_lasso_fert = GridSearchCV(
    Lasso(), params_linear, scoring="neg_mean_squared_error", cv=15, n_jobs=-1)

grid_search_lasso_fert.fit(X_fert_train, y_fert_train)
print(grid_search_lasso_fert.best_params_)
print("Melhores Parâmetros:", grid_search_lasso_fert.best_params_,"\n")
preds= grid_search_lasso_fert.predict(X_fert_test)
printRegStatistics(y_fert_test, preds)

{'alpha': 0.1}
Melhores Parâmetros: {'alpha': 0.1} 

The RVE is:  0.9977762259078578
The rmse is:  0.09277534454384653
The Correlation Score is is: 0.9993 (p-value=0.000000e+00)

The Maximum Error is is:  1.7453203082377713
The Mean Absolute Error is:  0.06721876638522725


In [29]:
# DecisionTreeRegressor
params =[{"max_depth" : [4,8,12,15,20]}]

grid_search_dtr_fert= GridSearchCV(
    DecisionTreeRegressor(), params, scoring="neg_mean_squared_error", cv=15, n_jobs=-1)


grid_search_dtr_fert.fit(X_fert_train, y_fert_train)
print(grid_search_dtr_fert.best_params_)
print("Melhores Parâmetros:", grid_search_dtr_fert.best_params_,"\n")
preds= grid_search_dtr_fert.predict(X_fert_test)
printRegStatistics(y_fert_test, preds)

{'max_depth': 12}
Melhores Parâmetros: {'max_depth': 12} 

The RVE is:  0.9989526790821193
The rmse is:  0.06365542727432057
The Correlation Score is is: 0.9995 (p-value=0.000000e+00)

The Maximum Error is is:  1.7751781798461543
The Mean Absolute Error is:  0.033993714942156285


In [30]:
# Melhor modelo eh o DecisionTreeRegressor
fertility_model = grid_search_dtr_fert

### Life Expectancy

Treino de 2 modelos, LinearRegression e DecisionTreeRegressor

In [31]:
# LinearRegression
params_linear = [
    {"alpha" : [x*0.01 for x in range(1,20)]}]

grid_search_lasso_exp = GridSearchCV(
    Lasso(), params_linear, scoring="neg_mean_squared_error", cv=15, n_jobs=-1)

grid_search_lasso_exp.fit(X_exp_train, y_exp_train)
print(grid_search_lasso_exp.best_params_)
print("Melhores Parâmetros:", grid_search_lasso_exp.best_params_,"\n")
preds= grid_search_lasso_exp.predict(X_exp_test)
printRegStatistics(y_exp_test, preds)

{'alpha': 0.01}
Melhores Parâmetros: {'alpha': 0.01} 

The RVE is:  0.9994918033284109
The rmse is:  0.23945833190789442
The Correlation Score is is: 0.9997 (p-value=0.000000e+00)

The Maximum Error is is:  3.2080776223917944
The Mean Absolute Error is:  0.11702207101954783


In [32]:
# DecisionTreeRegressor
params =[{"max_depth" : [4,8,12,15,20,25,30]}]

grid_search_dtr_exp = GridSearchCV(
    DecisionTreeRegressor(), params, scoring="neg_mean_squared_error", cv=15, n_jobs=-1)


grid_search_dtr_exp.fit(X_exp_train, y_exp_train)
print(grid_search_dtr_exp.best_params_)
print("Melhores Parâmetros:", grid_search_dtr_exp.best_params_,"\n")
preds= grid_search_dtr_exp.predict(X_exp_test)
printRegStatistics(y_exp_test, preds)

{'max_depth': 20}
Melhores Parâmetros: {'max_depth': 20} 

The RVE is:  0.9990560394239085
The rmse is:  0.3263052946105978
The Correlation Score is is: 0.9995 (p-value=0.000000e+00)

The Maximum Error is is:  5.3289999999999935
The Mean Absolute Error is:  0.16472345879316208


In [38]:
# Melhor modelo eh o DecisionTreeRegressor
expectancy_model = grid_search_lasso_exp

## Previsao de Valores de 2017

Escolher a sample de 10 países aleatórios

In [40]:
sample = list(fertility["Country Name"].sample(n=10, random_state=483))
print(sample)

['Lesotho', 'North America', 'New Zealand', 'Singapore', 'Europe & Central Asia', 'Qatar', 'Uzbekistan', 'Estonia', 'Angola', 'Japan']


### Populacao

In [41]:
PrevisionDataPop = []

for country in sample:
    x_prevision, delta_prevision = predictAndCalculateDelta(pop_model, pop_fin, country, 2017, lag=4)
    x_real, delta_real = determineRealResults(pop_2020, country, 2017)
    _, x_rel_error = calculateErrors(x_real, x_prevision)
    _, delta_rel_error = calculateErrors(delta_real, delta_prevision)
    data_entry =  (country, x_real, x_prevision, x_rel_error, delta_real, delta_prevision, delta_rel_error)
    PrevisionDataPop.append(data_entry)

# Criar dataframe
PopDF = DFCreate(PrevisionDataPop, 2017)
PopDF

,Country,2017 Real Value,2017 Prevision,Relative Error %,2017 Real Delta,2017 Delta Prevision,Relative Error %
0,Lesotho,"2,170,617.000","2,230,097.121",2.740,"26,745.000","26,276.121",1.753
1,North America,"361,731,237.000","362,420,608.218",0.191,"2,485,441.000","2,963,115.218",19.219
2,New Zealand,"4,813,600.000","4,739,639.551",1.536,"99,500.000","46,439.551",53.327
3,Singapore,"5,612,253.000","5,680,391.725",1.214,"4,970.000","73,108.725","1,371.001"
4,Europe & Central Asia,"915,855,416.000","916,360,611.459",0.055,"3,480,711.000","4,513,333.459",29.667
5,Qatar,"2,711,755.000","2,611,595.075",3.694,"116,589.000","41,791.075",64.155
6,Uzbekistan,"32,388,600.000","32,313,711.760",0.231,"540,700.000","465,811.760",13.850
7,Estonia,"1,317,384.000","1,315,548.143",0.139,"1,594.000",-241.857,115.173
8,Angola,"30,208,628.000","29,287,341.883",3.050,"1,053,882.000","473,878.883",55.035
9,Japan,"126,972,000.000","127,574,141.703",0.474,"-104,000.000","579,630.703",657.337


### Fertility Rate

In [42]:
PrevisionDataFert = []

for country in sample:
    x_prevision, delta_prevision = predictAndCalculateDelta(fertility_model, fert_fin, country, 2017, lag=4)
    x_real, delta_real = determineRealResults(fert_2020, country, 2017)
    _, x_rel_error = calculateErrors(x_real, x_prevision)
    _, delta_rel_error = calculateErrors(delta_real, delta_prevision)
    data_entry =  (country, x_real, x_prevision, x_rel_error, delta_real, delta_prevision, delta_rel_error)
    PrevisionDataFert.append(data_entry)

# Criar dataframe
FerDF = DFCreate(PrevisionDataFert, 2017)
FerDF

,Country,2017 Real Value,2017 Prevision,Relative Error %,2017 Real Delta,2017 Delta Prevision,Relative Error %
0,Lesotho,3.187,3.075,3.528,-0.052,-0.019,62.598
1,North America,1.743,1.764,1.204,-0.055,-0.016,70.750
2,New Zealand,1.810,1.837,1.479,-0.060,-0.033,44.619
3,Singapore,1.160,1.270,9.483,-0.040,0.070,275.000
4,Europe & Central Asia,1.735,1.734,0.068,-0.047,-0.016,64.814
5,Qatar,1.839,1.885,2.492,0.012,-0.022,284.818
6,Uzbekistan,2.419,2.434,0.616,-0.036,-0.021,41.395
7,Estonia,1.590,1.595,0.304,-0.010,0.015,248.264
8,Angola,5.600,5.614,0.250,-0.086,-0.080,6.977
9,Japan,1.430,1.446,1.109,-0.010,0.006,158.567


### Life Expectancy

In [43]:
PrevisionDataExp = []

for country in sample:
    x_prevision, delta_prevision = predictAndCalculateDelta(expectancy_model, exp_fin, country, 2017, lag=4)
    x_real, delta_real = determineRealResults(exp_2020, country, 2017)
    _, x_rel_error = calculateErrors(x_real, x_prevision)
    _, delta_rel_error = calculateErrors(delta_real, delta_prevision)
    data_entry =  (country, x_real, x_prevision, x_rel_error, delta_real, delta_prevision, delta_rel_error)
    PrevisionDataExp.append(data_entry)

# Criar dataframe
ExpDF = DFCreate(PrevisionDataExp, 2017)
ExpDF

,Country,2017 Real Value,2017 Prevision,Relative Error %,2017 Real Delta,2017 Delta Prevision,Relative Error %
0,Lesotho,53.064,54.668,3.023,0.800,0.494,38.215
1,North America,78.879,79.062,0.232,0.002,0.008,327.608
2,New Zealand,81.659,81.701,0.053,0.046,0.089,93.148
3,Singapore,83.095,82.939,0.187,0.249,0.144,41.989
4,Europe & Central Asia,77.779,77.376,0.518,0.213,0.094,56.159
5,Qatar,80.717,78.334,2.953,0.284,0.150,47.287
6,Uzbekistan,71.010,71.489,0.674,0.255,0.175,31.489
7,Estonia,78.093,77.997,0.122,0.451,0.260,42.273
8,Angola,61.680,61.934,0.411,0.588,0.387,34.260
9,Japan,84.100,84.158,0.070,0.115,0.173,50.983
